## Data Handling Script

**Purpose:**  (1) Extract necessary metrics from all multi-domain studies and export to `.csv` for further exploration, (2) calculate statistics on amount of data analyzed – **all with hmetad calculation**

Author: Saurish Srivastava @ [Subjectivity Lab](https://subjectivity.sites.northeastern.edu/)

In [ ]:
# install packages
!pip3 install numpy
!pip3 install pandas
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install scipy
!pip3 install arviz
!pip3 install numpyro
!pip3 install pymc
!pip3 install aesara
!pip3 install git+https://github.com/embodied-computation-group/metadPy.git

In [ ]:
# imports
import numpy as np
import pandas as pd
import arviz as az
import arviz as az
import numpy as np
import numpyro
import scipy.stats as st
from metadPy.bayesian import hmetad

from metadPy.mle import fit_metad, metad
from metadPy import load_dataset
from metadPy.utils import trials2counts
from metadPy.plotting import plot_confidence, plot_roc
from metadPy.sdt import scores, rates, dprime, criterion, roc_auc
import matplotlib.pyplot as plt


from functools import partial, reduce

# Set the number of cores used by Numpyro
numpyro.set_host_device_count(4)

## Define data handling functions

In [ ]:
def getGroupData(data, nRatings, export):
    """
    For dataframe {data} & int {nRatings} & string {export}: finds scores, rates, dprime, metad, 
    criterion, mratio, and mdiff for each subject and export it to {export} filepath.
    Returns: a dataframe with above variables formatted as {domain}_{variable} and exports to {export} location
    """
    # get domains
    domains = list(data['Task'].unique())
    
    # create temporary dictionary
    temp_dict = {}
    
    for domain in domains:
        domain_data = data.loc[data['Task'] == domain] # only get proper domain
        domain_data = domain_data.reset_index(drop=True) # reset indexes
        # iterate through each subject in the data
        temp_list = []
        print(f"{domain.capitalize()}\n------------")
        for i in list(domain_data['Subject'].unique()):
            temp_data = domain_data.loc[domain_data['Subject'] == i]
            # if metad function does not return error, continue with this subject
            try:
                model, traces = hmetad(
                    data=temp_data,
                    nRatings=nRatings,
                    stimuli="Stimuli",
                    accuracy="Accuracy",
                    confidence="Confidence",
                    subject="Subject",
                    backend="numpyro",
                )
            except:
                continue
            else:
                temp_dprime = az.summary(traces)['mean']['d1[0]']
                temp_criterion = az.summary(traces)['mean']['c1[0]']
                temp_metadprime = az.summary(traces)['mean']['meta_d[0]']
                temp_mratio = az.summary(traces)['mean']['m_ratio[0]']
                temp_logmratio = az.summary(traces)['mean']['log_m_ratio[0]']
                print(f"Done: Subject {i}")
                
                temp_list.append([i, temp_dprime, temp_criterion, temp_metadprime,
                                  temp_mratio, temp_logmratio])

        # create pandas dataframe from list data
        groupData = pd.DataFrame(temp_list, columns=['Index', domain + '_dprime', domain + '_criterion',
                                               domain + '_metad', domain + '_mratio', domain + '_log_mratio'])
        # change index name
        groupData = groupData.set_index('Index')
        groupData.index.name="Subject"
        
        # add to dictionary
        temp_dict[domain] = groupData
        
        print(f"------------\nCompleted: {domain.capitalize()}\n------------")
    
    # merge data
    my_reduce = partial(pd.merge, on='Subject', how='outer')
    fullData = reduce(my_reduce, temp_dict.values())
    
    # export data
    
    fullData.to_csv('../../exports/hmetad_' + export + '.csv')
    
    print(f"Exported: 'hmetad_{export}.csv'")
    
    return fullData

In [ ]:
# def getAnalyzedTrials(data, domains)
#     """
#     For dataframe 'data': find out the number of trials that are analyzed and the number of
#     trials that are excluded during metad calculations
#     Prints: # of subjects, # of recorded trials, # of analyzed and non-analyzed trials,
#     # of subjects not analyzed, and percentage accounted for & their difference in %
#     """
    
#     domains = list(data['Task'].unique())
#     temp_list = []
#     subjectCounter = 0
#     skipped_subject = []
#     for domain in domains: 
#         domain_data = data.loc[data['Task'] == domain] # only get proper domain
#         domain_data = domain_data.reset_index(drop=True) # reset indexes
        
#         # iterate through each subject in the data
#         for i in list(domain_data['Subject'].unique()):
#             temp_data = domain_data.loc[domain_data['Subject'] == i]
#             [nR_S1, nR_S2] = trials2counts(data=temp_data.copy(), stimuli="Stimuli", responses="Responses",
#                        confidence="Confidence", nRatings=nRatings, padding=True)
#             # if metad function does not return error, continue with this subject
#             try:
#                 fit_metad(nR_S1,nR_S2, nRatings=nRatings, nCriteria=int(2 * nRatings - 1))
#             except:
#                 skipped_subject.append("Subject " + str(i) + " @ " + domain)
#                 continue
#             else:
#                 subjectCounter += 1
#                 temp_list.append(temp_data.shape[0])
        
#         print("Completed: " + domain)

#     # calculate stats
#     percentageAccounted = 100*(sum(temp_list)/data.shape[0])
#     difference = 100 * (1 - (sum(temp_list)/data.shape[0]))
    
#     print("Number of subjects: " + str(subjectCounter))
#     print("Number of recorded trials: " + str(data.shape[0]))
#     print("Number of analyzed trials: " + str(sum(temp_list)))
#     print("Number of non-analyzed trials: " + str(int(data.shape[0] - sum(temp_list))))
#     print("Subjects not analyzed (@ domain): " + str(skipped_subject))
#     print("Percentage accounted for: " + str(percentageAccounted) + "%")
#     print("Difference (%): " + str(difference) + "%")

## Analysis
### `Arbuzova_unpub_3`

In [ ]:
# read in data
data = pd.read_csv("../../Confidence Database/data_Arbuzova_unpub_3.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [ ]:
getGroupData(data=data, nRatings=4, export='Arbuzova_unpub_3')

### `Mazancieux_2018`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Mazancieux_2018.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# add 1 to each confidence rating
data['Confidence'] = data['Confidence'] + 1

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [ ]:
getGroupData(data=data, nRatings=11, export='Mazancieux_2018')

### `Sadeghi_2017`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Sadeghi_2017_memory.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'group'])

data2 = pd.read_csv("../Confidence Database/data_Sadeghi_2017_perception.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'group'])

data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses", "group": "Task"})

data2 = data2.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses", "group": "Task"})


# get only control patients
data = data.loc[data['Task'] == 'control']
data2 = data2.loc[data2['Task'] == 'control']

# drop all NaNs
data = data.dropna().reset_index(drop=True)
data2 = data2.dropna().reset_index(drop=True)

# change task values according to the dataset
data['Task'] = 'memory'
data2['Task'] = 'perception'

# change perception dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data2['Stimuli'] = data2['Stimuli'].map({1:0, 2:1})
data2['Responses'] = data2['Responses'].map({1:0, 2:1})

# merge all data into just 'data' dataframe
data = data.append(data2)

In [ ]:
getGroupData(data=data, nRatings=6, export='Sadeghi_2017')

### `Samaha_2016`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Samaha_2016.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task', 'Condition', 'Accuracy'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's -45s->0s and 45s->1s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({-45:0, 45:1, 0:0, 1:1})
data['Responses'] = data['Responses'].map({-45:0, 45:1, 0:0, 1:1})

# rename tasks based on conditions - 0.5 -> low; 1 -> high
data['Task'] = np.where(data['Task'].eq("percept") & data['Condition'].eq(0.5), 'percept_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("percept") & data['Condition'].eq(1), 'percept_high', data['Task'])
data['Task'] = np.where(data['Task'].eq("wm") & data['Condition'].eq(0.5), 'wm_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("wm") & data['Condition'].eq(1), 'wm_high', data['Task'])

In [ ]:
getGroupData(data=data, nRatings=4, export='Samaha_2016')

### `Samaha_2017_exp3`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Samaha_2017_exp3.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's values bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({-1:0, 0:0, 1:1})
data['Responses'] = data['Responses'].map({-1:0, 0:0, 1:1})

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# get data in order by subject
data = data.sort_values(by=['Subject'])
data = data.reset_index(drop=True)

In [ ]:
getGroupData(data=data, nRatings=4, export='Samaha_2017_exp3')

### `Schmidt_2019`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Schmidt_2019_memory.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Accuracy', 'Condition'])

data2 = pd.read_csv("../Confidence Database/data_Schmidt_2019_perception.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Accuracy', 'Condition'])

data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses",
                            "Condition": "Task"})

data2 = data2.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses",
                              "Condition": "Task"})

# get only first 200 trials for memory bc they are 'pre-training'
data = data.groupby('Subject').head(200)

# get only first 300 trials for perception bc they are 'pre-training'
data2 = data2.groupby('Subject').head(300)

# drop all NaNs
data = data.dropna().reset_index(drop=True)
data2 = data2.dropna().reset_index(drop=True)

# change task values according to the dataset
data['Task'] = 'memory'
data2['Task'] = 'perception'

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

data2['Stimuli'] = data2['Stimuli'].map({1:0, 2:1})
data2['Responses'] = data2['Responses'].map({1:0, 2:1})

# merge all data into just 'data' dataframe
data = data.append(data2)

In [ ]:
getGroupData(data=data, nRatings=4, export='Schmidt_2019')

### `Skora_2016`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Skora_2016.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Condition', 'Accuracy'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses", "Condition": "Task"})

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [ ]:
getGroupData(data=data, nRatings=4, export='Skora_2016')

### `Xu_2019_Exp2`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Xu_2019_Exp2.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task', 'Condition'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# rename tasks based on conditions - 0.5 -> low; 1 -> high
data['Task'] = np.where(data['Task'].eq("N") & data['Condition'].eq("Incongruent"), 'N_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("N") & data['Condition'].eq("Congruent"), 'N_high', data['Task'])
data['Task'] = np.where(data['Task'].eq("C") & data['Condition'].eq("Incongruent"), 'C_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("C") & data['Condition'].eq("Congruent"), 'C_high', data['Task'])

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [ ]:
getGroupData(data=data, nRatings=4, export='Xu_2019_Exp2')

### `Ye_2018`

In [ ]:
# read in data
data = pd.read_csv("../Confidence Database/data_Ye_2018.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task', 'Accuracy'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [ ]:
getGroupData(data=data, nRatings=4, export='Ye_2018')